#Tratamento de dados
Este caderno tem por objetivo carregar os dados históricos e aplicar os tratamentos considerados necessários, tais como conversão de tipos e eliminação de dados considerados irrelevantes para a análise. Ao final, os dados consolidados são salvos em um arquivo para processamento posterior.

##Operações:
1. Removidas observações com ‘Status’ igual a ‘Registro’
1. As colunas ‘Protocolo’ e ‘Registro’ são convertidas para tipo data
2. Criada a coluna ‘Mês_ocorr’ com o mês da ocorrência do desastre
3. Criada a coluna ‘Atraso’ com a quantidade de dias entre a ocorrência e o registro do desastre
4. Conversão das categorias nas colunas de Danos Ambientais para tipos numéricos com pesos apropriados
6. Eliminação das colunas ‘Protocolo’, ‘UF’, ‘Município’ e ‘Registro’
7. Eliminação de registros com valores ausentes


##*Bibliotecas*

In [ ]:
from glob import glob
import pandas as pd
import numpy as np
from datetime import datetime

##Carregando dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Aplicando transformação de dados arquivo a arquivo

In [ ]:
# files_path = glob('/path/to/data/Danos_Informados_*.xls')
# print(files_path)
def extract_transform(file_path):
    # Abrindo e lendo os dados do arquivo
    df_danos = pd.read_excel(file_path,
                            skiprows=4,
                            converters={'Registro': lambda x: datetime.strptime(x,'%d/%m/%Y'),
                                        'Protocolo': lambda x: datetime.strptime(x[-6:],'%y%m%d'),
                                        'COBRADE': lambda x: x[:5]
                                        },
                            )
    # Conversão das colunas DA
    encoding = {np.nan:0, 'DE 0% A 5%':2.5, 'DE 5% A 10%':7.5, 'DE 10% A 20%':15, 'ATÉ 40%':22.5,'MAIS DE 20%':60, 'MAIS DE 40%':70}
    da_columns = [da for da in df_danos if da.startswith('DA_')]
    for da in da_columns:
        df_danos[da] = df_danos[da].map(encoding)
    df_danos['Atraso'] = (df_danos['Registro'] - df_danos['Protocolo']).apply(lambda x: x.days)
    df_danos['Mes_ocorr'] = df_danos['Registro'].apply(lambda x: x.strftime('%b'))
    return df_danos[~(df_danos['Status'] == 'Registro')].iloc[:,[0]+list(range(4,len(df_danos.columns)))]
  

Consolidando dados em um único objeto do tipo pandas.DataFrame

In [ ]:
# importar agora todos os arquivos xls e concatenar em um dataframe
# all_data = []
# for file in files_path:
#   all_data.append(extract_transform(file))
# Concatenando dados em um único objeto de dados
df_danos = extract_transform('/path/to/data/Danos_Informadosde2021.xls')
# Liberando memória
# del all_data

Apresentando informações de colunas e tipos para verificação

In [ ]:
print(df_danos.shape)
df_danos.info()

(1014, 52)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 383 to 1855
Data columns (total 52 columns):
 #   Column                                                                                  Non-Null Count  Dtype  
---  ------                                                                                  --------------  -----  
 0   UF                                                                                      1014 non-null   object 
 1   COBRADE                                                                                 1014 non-null   object 
 2   Status                                                                                  1014 non-null   object 
 3   População                                                                               1014 non-null   int64  
 4   DH_Mortos                                                                               1014 non-null   int64  
 5   DH_Feridos                                              

##Identificando e removendo observações com dados faltantes

In [ ]:
df_danos[df_danos.isna().any(axis=1)]

,UF,COBRADE,Status,População,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor,DM_Instalações públicas de saúde Danificadas,DM_Instalações públicas de saúde Destruídas,DM_Instalações públicas de saúde Valor,DM_Instalações públicas de ensino Danificadas,DM_Instalações públicas de ensino Destruídas,DM_Instalações públicas de ensino Valor,DM_Instalações públicas prestadoras de outros serviços Danificadas,DM_Instalações públicas prestadoras de outros serviços Destruídas,DM_Instalações públicas prestadoras de outros serviços Valor,DM_Instalações públicas de uso comunitário Danificadas,DM_Instalações públicas de uso comunitário Destruídas,DM_Instalações públicas de uso comunitário Valor,DM_Obras de infraestrutura pública Danificadas,DM_Obras de infraestrutura pública Destruídas,DM_Obras de infraestrutura pública Valor,DA_Poluição ou contaminação da água,DA_Poluição ou contaminação do ar,DA_Poluição ou contaminação do solo,DA_Diminuição ou exaurimento hídrico,"DA_Incêndios em parques, APA's ou APP's","PEPL_Assistência médica, saúde pública e atendimento de emergências médicas (R$)",PEPL_Abastecimento de água potável (R$),PEPL_Esgoto de águas pluviais e sistema de esgotos sanitários (R$),PEPL_Sistema de limpeza urbana e de recolhimento e destinação do lixo (R$),PEPL_Sistema de desinfestação/desinfecção do habitat/controle de pragas e vetores (R$),PEPL_Geração e distribuição de energia elétrica (R$),PEPL_Telecomunicações (R$),"PEPL_Transportes locais, regionais e de longo curso (R$)","PEPL_Distribuição de combustíveis, especialmente os de uso doméstico (R$)",PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),Atraso,Mes_ocorr


In [ ]:
print(df_danos.shape)
# df_danos.drop(index=df_danos[df_danos.isna().any(axis=1)].index,
#               inplace=True)
df_danos.dropna(inplace=True)
print(df_danos.shape)

(28349, 52)
(28342, 52)


##Salvando arquivo de dados consolidado

In [ ]:
df_danos.to_excel('/path/to/data/Danos_2021.xls',index=False)